In [1]:
!pip install z3-solver
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 91 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 8.4 MB/s 


In [2]:
from google.colab import drive
from google.colab import files
from z3 import *
import matplotlib.pyplot as plt
import numpy as np
import itertools
import xlsxwriter

In [3]:
INSTANCE_PATH = '/content/gdrive/MyDrive/unibo/CombinatorialProject/VLSI_instances/'
RESULT_PATH = "/content/gdrive/MyDrive/unibo/CombinatorialProject/SAT_Res/"

In [4]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
def read_instance(n):
    with open(INSTANCE_PATH + 'ins-' + str(n) + '.txt') as f:
        contents = f.read()
    f.close()

    contents = contents.splitlines()
    w = int(contents[0])
    n = int(contents[1])
    mtr = []
    for i in range(2, n + 2):
        mtr.append([int(x) for x in contents[i].split()])

    return {'w': w, 'n': n, 'mtr': mtr}


def write_results(num_instance, w, h, n, widths, heights, corners_x, corners_y, rotation = False, rotated = []):
    path =''
    if rotation:
      path = RESULT_PATH + 'sol-rot-' + str(num_instance) + '.txt'
    else:
      path = RESULT_PATH + 'sol-' + str(num_instance) + '.txt'

    with open(path, 'w') as f:
        f.write(str(w) + " " + str(h) + "\n")
        f.write(str(n) + '\n')
        for i in range(n):
            if rotation and rotated[i]:
               f.write(str(heights[i]) + " " + str(widths[i]) + " "
                    +  str(corners_x[i]) + " " + str(corners_y[i]) + "\n")
            else:
              f.write(str(widths[i]) + " " + str(heights[i]) + " "
                      +  str(corners_x[i]) + " " + str(corners_y[i]) + "\n")
    f.close()


def show_results(num_instance, rotation=False, rotated=None):
    path = ''
    if rotation:
        path = RESULT_PATH  + 'sol-rot-' + str(num_instance) + '.txt'
    else:
        path = RESULT_PATH  + 'sol-' + str(num_instance) + '.txt'

    with open(path) as f:
        contents = f.read()
    f.close()

    contents = contents.splitlines()
    w, h = contents[0].split()
    n = int(contents[1])
    w = int(w)
    h = int(h)

    widths = []
    heights = []
    corners_x = []
    corners_y = []

    for k in range(2, n + 2):
        width, height, x, y = contents[k].split()
        widths.append(int(width))
        heights.append(int(height))
        corners_x.append(int(x))
        corners_y.append(int(y))

    title = 'Solution instance ' + str(num_instance)
    if rotation:
        title += ' with rotation'

    fig, ax = plt.subplots()
    ax.set_title(title)

    for i in range(n):
        color = np.array(np.random.choice(range(30, 231), size=3))
        color = color / 255
        rect = plt.Rectangle((corners_x[i], corners_y[i]), widths[i], heights[i],
                             facecolor=color, edgecolor='black', linestyle='solid', linewidth=1.5)
        ax.add_patch(rect)

        if rotation and rotated[i]:
            circle = plt.Circle((corners_x[i] + widths[i] / 2, corners_y[i] + heights[i] / 2), 0.20, color='w',zorder=10)
            ax.add_patch(circle)

    ax.set(xlim=(0, w), ylim=(0, h))
    ax.set_aspect('equal')
    ax.grid(color='black', linewidth=0.75, linestyle='--')

    plt.xticks(np.arange(0, w + 1, 1), rotation=90)
    plt.yticks(np.arange(0, h + 1, 1))

    path = ''
    if rotation:
        path = RESULT_PATH + 'img/sol-rot-' + str(num_instance) + '.png'
    else:
        path = RESULT_PATH + 'img/sol-' + str(num_instance) + '.png'
    plt.savefig(path)
    plt.show()
    plt.close()

"""Absolute value function for Z3 solver, order by area function for block"""
def order_by_area(shape_matrix):
    n = len(shape_matrix[:, 0])
    for i in range(n - 1):
        for j in range(i + 1, n):
            if (shape_matrix[i, 0] * shape_matrix[i, 1]) < (shape_matrix[j, 0] * shape_matrix[j, 1]):
                tmp = np.copy(shape_matrix[i])
                shape_matrix[i] = np.copy(shape_matrix[j])
                shape_matrix[j] = np.copy(tmp)

    return shape_matrix

def get_lower_bound(instance, rotation = False):
    w = instance['w']
    n = instance['n']
    shape_matrix = instance['mtr']
    min_h = 0
    for k in range(n):
        min_h += shape_matrix[k][0] * shape_matrix[k][1]
    min_h = np.ceil(min_h / w)

    if rotation:
      min_between_h_w = np.min([np.max(shape_matrix[:, 1]), np.max(shape_matrix[:, 0])])
      return int(np.max([min_h, min_between_h_w]))
    else:
      return int(np.max([min_h, np.max(shape_matrix[:, 1])]))


def get_upper_bound(instance):
  w = instance['w']
  n = instance['n']
  shape_matrix = instance['mtr']

  combination_indexes = []
  combination_indexes.append(0)

  width_sum = shape_matrix[0][0]
  width_sum_no_last = 0

  for i in range(1, n):
    width_sum_no_last = width_sum
    width_sum += shape_matrix[i][0]
   
    if( ( (width_sum - 1 ) % w) < ( (width_sum_no_last - 1 )% w) ) :
      combination_indexes.append(i)

  combination_indexes.append(n-1)

  max_h = 0
  for i in range (len(combination_indexes) -1):
    if combination_indexes[i] != combination_indexes[i+1]:
      max_h += np.max(shape_matrix[combination_indexes[i]:(combination_indexes[i+1]),1])
    else:
      max_h += shape_matrix[combination_indexes[i],1]
    

  return int(max_h)


def prepare_instance(instance_num):
    instance = read_instance(instance_num)
    instance['mtr'] = order_by_area(np.asarray(instance['mtr']))
    return instance



In [6]:
def compute_statistics(rotation, symmetry_breaking ,all_num_instances, all_solutions, all_is_optimal_solution, all_statistics, count):

  title = '/content/gdrive/MyDrive/unibo/CombinatorialProject/' + str(count) +'-SAT_default'
  propagations = 0
  conflicts = None

  if rotation:
    title += '_With_rotation'
  else:
    title += '_no_rotation'
  
  if symmetry_breaking:
    title += '_with_symmetry_breaking'
  else:
    title += '_no_symmetry_breaking'

  title += '.xlsx'

  print(title)

  workbook = xlsxwriter.Workbook(title)
  worksheet = workbook.add_worksheet()

  # Add a bold format to use to highlight cells.
  bold = workbook.add_format({'bold': True})
  cell_format_red_bg = workbook.add_format({'bg_color': 'red'})
 
  # Write some data headers.
  worksheet.write('A1', 'Instance', bold)
  worksheet.write('B1', 'Time', bold)
  worksheet.write('C1', 'Conflicts', bold)
  worksheet.write('D1', 'Propagations', bold)
  worksheet.write('E1', 'Best solution', bold)

  start_row =  1
  for i in range(len(all_num_instances)):
    worksheet.write(start_row +i, 0, all_num_instances[i])

    if all_is_optimal_solution[i]:
      time = round(all_statistics[i].get_key_value('time'), 3)
      worksheet.write(start_row +i, 1, time)
      worksheet.write(start_row +i, 4, all_solutions[i])
    else:
      worksheet.write(start_row +i, 1, 'Timeout', cell_format_red_bg)
      worksheet.write(start_row +i, 4, all_solutions[i], cell_format_red_bg)

    if all_statistics[i] is not None:
      if  'sat conflicts' in all_statistics[i].keys():
        conflicts = int(all_statistics[i].get_key_value('sat conflicts'))
        worksheet.write(start_row +i, 2, conflicts)
      else:
        worksheet.write(start_row +i, 2, '-')
      if 'sat propagations 2ary' in all_statistics[i].keys():
        propagations += int(all_statistics[i].get_key_value('sat propagations 2ary'))
      if 'sat propagations 3ary' in all_statistics[i].keys():
        propagations += int(all_statistics[i].get_key_value('sat propagations 3ary'))
      if 'sat propagations nary' in all_statistics[i].keys():
        propagations += int(all_statistics[i].get_key_value('sat propagations nary'))  
    

    if propagations != 0:
      worksheet.write(start_row +i, 3, propagations)
    else:
      worksheet.write(start_row +i, 3, '-')

  workbook.close()

In [7]:

def SAT_encoding(instance, h, symmtry_breaking = False):
  w = instance['w']
  widths = instance['mtr'][:,0]
  heights = instance['mtr'][:,1]
  n = instance['n']

  g = Goal()

  # px and py are used to apply order encoding and they encode the fact that a block can have as first or second coordinate
  # the value of the second index.
  # e.g. px(i,z) means that the x-coordinate of the block i can be z.
  px = [[Bool(f"px{i+1}_{x}") for x in range(w)] for i in range(n)]
  py = [[Bool(f"py{i+1}_{y}") for y in range(h)] for i in range(n)]

  # Under and left are two matrices representing the fact that the block is below or at the left of another block
  # e.g. under(i,j) represents that block i is below block j. (Same thing for left)
  under = [[Bool(f"ud_{i+1}_{j+1}") if i != j else 0 for j in range(n) ] for i in range(n)]
  left = [[Bool(f"lt{i+1}_{j+1}")  if j != i else 0 for j in range(n)] for i in range(n)]


  # Each pair of block cannot overlap
  for  i in range(n):
    for j in range(i+1, n):
      g.add(Or( left[i][j], left[j][i], under[i][j], under[j][i]))
  
  # Clauses due to order ecoding
  for i in range(n):
    for e in range(0, w - widths[i]):
      g.add(Or(Not(px[i][e]), px[i][e+1]))

    for f in range(0, h - heights[i]):
      g.add(Or(Not(py[i][f]), py[i][f+1]))
  
    # Implicit clauses also due to order encoding
    for e in range(w-widths[i],w):
      g.add(px[i][e])
  
    for f in range(h-heights[i],h):
      g.add(py[i][f])
  

  for i in range(n):
    for j in range(n):
      if i!= j:
        # left(i,j) -> xj > wi, lower bound for xj
        g.add(Or( Not(left[i][j]), Not(px[j][widths[i]-1]))) 
        #under(ri,rj)-> yj > hi, lower bound for yj
        g.add(Or(Not(under[i][j]),Not(py[j][heights[i]-1]))) 

        # 3-literals clauses for non overlapping, shown in the paper
        for e in range(0,w-widths[i]):
          g.add(Or(Not(left[i][j]), px[i][e], Not(px[j][e+widths[i]])))
        for e in range(0,w-widths[j]):
          g.add(Or(Not(left[j][i]), px[j][e], Not(px[i][e+widths[j]])))
 
        for f in range(0,h-heights[i]):
          g.add(Or(Not(under[i][j]), py[i][f], Not(py[j][f+heights[i]])))
        for f in range(0,h-heights[j]):
          g.add(Or(Not(under[j][i]), py[j][f], Not(py[i][f+heights[j]])))

  if symmtry_breaking:
    g.add(And(use_symmetry_breaking(n,w,h,widths,heights,px,py,left,under)))
  
  return g

In [8]:
def use_symmetry_breaking(n,w, h, widths, heights, px, py, left, under):
  domain_reduction_constraint = []
  # domain reduction constraint
  for i in range(int(np.floor((w-widths[0])/2))):
      domain_reduction_constraint.append(Not(px[0][i]))
  
  for j in range(int(np.floor((h-heights[0])/2))):
      domain_reduction_constraint.append(Not(py[0][j]))

  for rect in range(1,n):
    if widths[rect] > np.ceil((w-widths[0])/2):
      domain_reduction_constraint.append(Not(left[0][rect]))
    if heights[rect] > np.ceil((h-heights[0])/2):
      domain_reduction_constraint.append(Not(under[0][rect]))


  # Same size rectangles
  for (i,j) in itertools.combinations(range(n),2):
    if widths[i] == widths[j] and heights[i] == heights[j]:
      domain_reduction_constraint.append(Not(left[j][i]))
      domain_reduction_constraint.append(
          Or(Not(under[j][i]), left[i][j])
      )

  return domain_reduction_constraint

In [9]:
def SAT_encoding_rotation(instance, h, symmtry_breaking = False):
  w = instance['w']
  widths = instance['mtr'][:,0]
  heights = instance['mtr'][:,1]
  n = instance['n']

  g = Goal()

  # px and py are used to apply order encoding and they encode the fact that a block can have as first or second coordinate
  # the value of the second index.
  # e.g. px(i,z) means that the x-coordinate of the block i can be z.
  px = [[Bool(f"px{i+1}_{x}") for x in range(w)] for i in range(n)]
  py = [[Bool(f"py{i+1}_{y}") for y in range(h)] for i in range(n)]

  # Under and left are two matrices representing the fact that the block is below or at the left of another block
  # e.g. under(i,j) represents that block i is below block j. (Same thing for left)
  under = [[Bool(f"ud_{i+1}_{j+1}") if i != j else 0 for j in range(n) ] for i in range(n)]
  left = [[Bool(f"lt{i+1}_{j+1}")  if j != i else 0 for j in range(n)] for i in range(n)]
  
  rotated = [Bool(f"r_{i+1}") for i in range(n)]

  # Each pair of block cannot overlap
  for  i in range(n):
    for j in range(i+1, n):
      g.add(Or( left[i][j], left[j][i], under[i][j], under[j][i]))
  
  # Clauses due to order ecoding
  for i in range(n):

    if heights[i] <= w:
        g.add(Or(And(Not(rotated[i]), *[Or(Not(px[i][e]), px[i][e+1]) for e in range(0, w - widths[i])]),
                And(rotated[i], *[Or(Not(px[i][e]), px[i][e+1]) for e in range(0, w - heights[i])])))

        g.add(Or(And(Not(rotated[i]), *[Or(Not(py[i][f]), py[i][f+1]) for f in range(0, h - heights[i])]),
                And(rotated[i], *[Or(Not(py[i][f]), py[i][f+1]) for f in range(0, h - widths[i])])))
        
        # Implicit clauses also due to order encoding
        g.add(Or(And(Not(rotated[i]), *[ px[i][e] for e in range(w-widths[i], w)]),
                And(rotated[i], *[ px[i][e] for e in range(w-heights[i], w)])))
        
        g.add(Or(And(Not(rotated[i]), *[ py[i][f] for f in range(h-heights[i], h)]),
                And(rotated[i], *[ py[i][f] for f in range(h-widths[i], h)])))
    else:
        g.add(Not(rotated[i]))
        g.add( *[Or(Not(px[i][e]), px[i][e+1]) for e in range(0, w - widths[i])])
        g.add( *[Or(Not(py[i][f]), py[i][f+1]) for f in range(0, h - heights[i])])
        # Implicit clauses also due to order encoding
        g.add(*[ px[i][e] for e in range(w-widths[i], w)])
        g.add(*[ py[i][f] for f in range(h-heights[i], h)])
        

  
  # (original_width = original_height) ==> rotated == False    
  g.add([ Not(rotated[i]) for i in range(n) if widths[i] == heights[i]])
  # original_height > plate_width ==> rotated == False  
  g.add([ Not(rotated[i]) for i in range(n) if heights[i] > w])

  # original_width > plate_height ==> rotated == False
  g.add([ Not(rotated[i]) for i in range(n) if widths[i] > h])


  for i in range(n):
    for j in range(n):
      if i!= j:

       
        if heights[i] <= w :
          g.add(Or(And(Not(rotated[i]), Or( Not(left[i][j]), Not(px[j][widths[i]-1]))),
                 And(rotated[i], Or( Not(left[i][j]), Not(px[j][heights[i]-1])))))
          
          
          #under(ri,rj)-> yj > hi, lower bound for yj
          g.add(Or(And(Not(rotated[i]),Or(Not(under[i][j]),Not(py[j][heights[i]-1]))),
                 And(rotated[i], Or(Not(under[i][j]),Not(py[j][widths[i]-1])))))

          # 3-literals clauses for non overlapping, shown in the paper
          g.add(Or(And(Not(rotated[i]), *[Or(Not(left[i][j]), px[i][e], Not(px[j][e+widths[i]])) for e in range(0, w - widths[i])]),
                  And(rotated[i], *[Or(Not(left[i][j]), px[i][e], Not(px[j][e+heights[i]])) for e in range(0, w - heights[i])])))
                
          g.add(Or(And(Not(rotated[j]), *[Or(Not(left[j][i]), px[j][e], Not(px[i][e+widths[j]])) for e in range(0, w - widths[j])]),
                  And(rotated[j], *[Or(Not(left[j][i]), px[j][e], Not(px[i][e+heights[j]])) for e in range(0, w - heights[j])])))
          

          g.add(Or(And(Not(rotated[i]), *[Or(Not(under[i][j]), py[i][f], Not(py[j][f+heights[i]])) for f in range(0, h - heights[i])]),
                  And(rotated[i], *[Or(Not(under[i][j]), py[i][f], Not(py[j][f+widths[i]])) for f in range(0, h - widths[i])])))
      
          g.add(Or(And(Not(rotated[j]), *[Or(Not(under[j][i]), py[j][f], Not(py[i][f+heights[j]])) for f in range(0, h - heights[j])]),
                  And(rotated[j], *[Or(Not(under[j][i]), py[j][f], Not(py[i][f+widths[j]])) for f in range(0, h - widths[j])])))

        else:
          # left(i,j) -> xj > wi, lower bound for xj
          g.add(Or( Not(left[i][j]), Not(px[j][widths[i]-1])))
          #under(ri,rj)-> yj > hi, lower bound for yj
          g.add(Or(Not(under[i][j]),Not(py[j][heights[i]-1])))
          # 3-literals clauses for non overlapping, shown in the paper
          g.add(*[Or(Not(left[i][j]), px[i][e], Not(px[j][e+widths[i]])) for e in range(0, w - widths[i])])

          g.add(*[Or(Not(left[j][i]), px[j][e], Not(px[i][e+widths[j]])) for e in range(0, w - widths[j])])

          g.add(*[Or(Not(under[i][j]), py[i][f], Not(py[j][f+heights[i]])) for f in range(0, h - heights[i])])

          g.add(*[Or(Not(under[j][i]), py[j][f], Not(py[i][f+heights[j]])) for f in range(0, h - heights[j])])

    # Large rectangle constraints from the paper
  for (i,j) in itertools.combinations(range(n),2):
    if widths[i] + widths[j] > w:
      g.add(Not(left[i][j]))
      g.add(Not(left[j][i]))
    if heights[i] + heights[j] > h:
      g.add(Not(under[i][j]))
      g.add(Not(under[j][i]))

  if symmtry_breaking:
    print('- add sb constraints')
    g.add(And(use_symmetry_breaking_rotation(n,w,h,widths,heights,px,py,left,under, rotated)))
  
  return g

In [10]:
def use_symmetry_breaking_rotation(n,w, h, widths, heights, px, py, left, under, rotated):
  domain_reduction_constraint = []
  # domain reduction constraint

  if heights[0] <= w:
    domain_reduction_constraint.append(Or(And( rotated[0], *[Not(px[0][i]) for i in range(int(np.floor((w-heights[0])/2)))] ),
                                          And( Not(rotated[0]), *[Not(px[0][i]) for i in range(int(np.floor((w-widths[0])/2)))])))
    
    domain_reduction_constraint.append(Or(And( rotated[0], *[Not(py[0][j]) for j in range(int(np.floor((h-widths[0])/2)))] ),
                                          And( Not(rotated[0]), *[Not(py[0][j]) for j in range(int(np.floor((h-heights[0])/2)))])))
  else:
      domain_reduction_constraint.append(And(*[Not(px[0][i]) for i in range(int(np.floor((w-widths[0])/2)))]))
    
      domain_reduction_constraint.append(And(*[Not(py[0][j]) for j in range(int(np.floor((h-heights[0])/2)))]))

  return domain_reduction_constraint

In [11]:
def SAT_check(sat_encoding, time_limit = None):

  t = Then(
        Repeat(With(Tactic('simplify'), cache_all = True)),
        Repeat('propagate-values'),
        'symmetry-reduce',
        'sat-preprocess'
     )

  theory_solver = Tactic('psat')
  final_tactic = Then(t, theory_solver)


  s = final_tactic.solver()
  
  s.add(sat_encoding)
  
  if time_limit is not None:
      print('Solver timout: ', time_limit, 's')
      s.set("timeout", int(time_limit * 1000))
  
  if s.check() == sat:
    return True, s.model(), s.statistics()

  else:
    return False, None, s.statistics()


def converter_sat_coord(m, w, h, n, rotation = False):
  px = [[Bool(f"px{i+1}_{x}") for x in range(w)] for i in range(n)]
  py = [[Bool(f"py{i+1}_{y}") for y in range(h)] for i in range(n)]

  rotated_vars = [Bool(f"r_{i+1}") for i in range(n)]
  rotated = []

  x_sol = []
  y_sol = [] 

  for i in range(n):
    if rotation:
      if m.evaluate(rotated_vars[i]) == True:
        rotated.append(True)
      else:
        rotated.append(False)

    j = 0
    while j < w:
      if m.evaluate(px[i][j]):
        x_sol.append(j)
        break
      j += 1

    j = 0
    while j < h:
      if m.evaluate(py[i][j]):
        y_sol.append(j)
        break
      j += 1

  return x_sol, y_sol, rotated

In [12]:
def solve_instances(instance_num, time_limit = None, rotation = False, symmetry_breaking = False, verbose = False, count = 0):
  all_num_instances = []
  all_solutions = []
  all_is_optimal_solution = []
  all_statistics = []

  for i in instance_num:
    print('\n***Instance num: ', str(i), '***\n')

    all_num_instances.append(i)
    instance = prepare_instance(i)
    min_h = get_lower_bound(instance, rotation)
    max_h = get_upper_bound(instance)
    
    print('Plate width -> ', instance['w'])
    print('Min h -> ', min_h)
    print('Max h -> ', max_h)

    count_attempt = 1;
    test_h = min_h
    model = None
    statistics = None
    time = 0

    while (min_h <= test_h and test_h <= max_h):
      print('\n- Attempt num -> ', count_attempt)
      print(' - Tested h -> ', test_h)

      if rotation:
        print('- rotation enabled')
        encoding = SAT_encoding_rotation(instance, test_h, symmetry_breaking)
      else:
        encoding = SAT_encoding(instance, test_h, symmetry_breaking)


      if time_limit is not None:
        is_sat, model_tmp, statistics_tmp = SAT_check(encoding, time_limit - time)
      else: 
        is_sat, model_tmp, statistics_tmp = SAT_check(encoding, None)

  
      time_tmp = statistics_tmp.get_key_value('time')
      time += time_tmp

      print('Expired time for this attempt -> ', time_tmp,'s')
      if is_sat != True:
        print('**UNSAT**')
        test_h +=1

      else:
        print('**SAT**')
        model = model_tmp
        statistics = statistics_tmp
        break

      count_attempt += 1

      if time_limit is not None and time >= time_limit:
        break
        
    if (time_limit is None or time <= time_limit) and model != None:
      print('Optimal solution founded in ', round(time, 3), 's, h -> ', test_h)

      all_is_optimal_solution.append(True)
      all_solutions.append(test_h)

      if verbose and statistics is not None:
          print('Statistics from z3 solver')
          print(statistics)
      
      all_statistics.append(statistics)

      corner_x, corner_y, rotated = converter_sat_coord(model, instance['w'], test_h , instance['n'], rotation)
      write_results(i,instance['w'],test_h,instance['n'],instance['mtr'][:,0],instance['mtr'][:,1],corner_x,corner_y, rotation, rotated)
      show_results(i,rotation, rotated)
        
    else:
      print('No optimal solution in' + str(time_limit) + 's founded')
      all_is_optimal_solution.append(False)
      

      if model != None:
        print('Last suboptimal solution')

        if verbose and statistics is not None:
          print('Statistics from z3 solver')
          print(statistics)
        
        all_statistics.append(statistics)
        all_solutions.append(test_h)
        corner_x, corner_y, rotated = converter_sat_coord(model, instance['w'], test_h, instance['n'], rotation)
        write_results(i,instance['w'],test_h,instance['n'],instance['mtr'][:,0],instance['mtr'][:,1],corner_x,corner_y, rotation, rotated)
        show_results(i, rotation, rotated)

      else:
        all_statistics.append(None)
        all_solutions.append(0)

  compute_statistics(rotation, symmetry_breaking ,all_num_instances, all_solutions, all_is_optimal_solution, all_statistics, count)



In [14]:
time_limit = 300
instance_num =np.linspace(1, 39, 39, dtype=int)

In [ ]:
number_tests = 5
for i in range(number_tests):
   solve_instances(instance_num, time_limit, rotation = False, symmetry_breaking = False, verbose = False, count = i+1)
   solve_instances(instance_num, time_limit, rotation = False, symmetry_breaking = True, verbose = False, count = i+1)
   solve_instances(instance_num, time_limit, rotation = True, symmetry_breaking = False, verbose = False, count = i+1)
   solve_instances(instance_num, time_limit, rotation = True, symmetry_breaking = True, verbose = False, count = i+1)